<a href="https://colab.research.google.com/github/ssvadla/Research_01/blob/main/Supervised_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

train = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train.head()

Mounted at /content/drive


Unnamed: 0                                           Sentence   Target
0         659  Appellant had stated to the officers that she ...  Invalid
1        3456  We shall discuss the facts more fully in conne...   Others
2        2043  â€œPerjury is a false statement, either writte...  Invalid
3        3344  The offense is felony theft by false pretext; ...    Issue
4        3231  Numerous contentions urging the commission of ...    Issue

In [2]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [3]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(3476, 6383)


In [5]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [6]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm


X, y = make_classification(random_state=1)

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
classifier = svm.SVC(decision_function_shape='ovo')
#classifier = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(x_val)
#print(np.unique(y_pred))
#print(type(y_pred))

Accuracy_score = accuracy_score(y_val, y_pred)

print('Validation Accuracy_score: %f' % Accuracy_score)
test_pred = classifier.predict(t_p)
acc_test = accuracy_score(test['Target'],test_pred)
print('Accuracy test data', acc_test)
classification_report = classification_report(test['Target'],test_pred)
print(classification_report)
matrix = confusion_matrix( y_val, y_pred)
print(matrix)


Validation Accuracy_score: 0.642699
Accuracy test data 0.6142034548944337
                  precision    recall  f1-score   support

        Analysis       0.60      0.19      0.29        77
      Conclusion       0.93      0.50      0.65        26
           Facts       0.60      0.97      0.74       267
         Invalid       0.74      0.28      0.40        83
           Issue       0.60      0.09      0.15        34
Rule/Law/Holding       0.50      0.21      0.29        34

        accuracy                           0.61       521
       macro avg       0.66      0.37      0.42       521
    weighted avg       0.63      0.61      0.55       521

[[ 27   0  94   4   1   0]
 [  4  16  24   1   1   0]
 [  5   0 475   2   2   1]
 [  5   0  88  37   0   2]
 [  2   0  31   2  14   2]
 [  4   0  42   1   5  12]]
